## 智能问答，相关设计

#### 1、数据清洗和规范化，
1.1转换为json格式

In [2]:
import pandas as pd
import jieba
import json

# 读取数据
data = pd.read_excel("校园新闻.xlsx", sheet_name="Sheet1")

# 清理空白和重复行
data = data.dropna().drop_duplicates()

# 基于标题生成简单问题
def generate_question(row):
    title = row["新闻标题"]
    question = f"{title} 是啥情况？"
    return question

# 生成问题列
data["生成的问题"] = data.apply(generate_question, axis=1)

# 使用 jieba 对问题和内容进行分词
data["分词问题"] = data["生成的问题"].apply(lambda x: " ".join(jieba.cut(x)))
data["分词答案"] = data["新闻内容"].apply(lambda x: " ".join(jieba.cut(x)))

# 转换为适合模型训练的 JSON 格式
train_data = data[["分词问题", "分词答案"]].rename(columns={"分词问题": "question", "分词答案": "answer"})
train_data_list = train_data.to_dict(orient="records")

# 将训练数据保存为 JSON 文件
with open("train_data.json", "w", encoding="utf-8") as f:
    json.dump(train_data_list, f, ensure_ascii=False, indent=4)

print("训练数据已成功保存为 train_data.json 格式（分词处理）")


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\blhq\AppData\Local\Temp\jieba.cache
Loading model cost 0.912 seconds.
Prefix dict has been built successfully.


训练数据已成功保存为 train_data.json 格式（分词处理）


#### 1.2设置截断词汇
效果一般

In [4]:
import pandas as pd
import jieba
import json

# 读取数据
data = pd.read_excel("校园新闻.xlsx", sheet_name="Sheet1")

# 清理空白和重复行
data = data.dropna().drop_duplicates()

# 基于标题生成简单问题
def generate_question(row):
    title = row["新闻标题"]
    question = f"{title} 是啥情况？"
    return question

# 生成问题列
data["生成的问题"] = data.apply(generate_question, axis=1)

# 定义一个函数在遇到 "供稿" 时截断新闻内容
def truncate_content(content):
    stop_word = "（供稿"
    truncated_content = content.split(stop_word)[0]  # 截断“供稿”之后的内容
    return truncated_content

# 先对新闻内容进行截断，再分词
data["截断内容"] = data["新闻内容"].apply(truncate_content)
data["分词答案"] = data["截断内容"].apply(lambda x: " ".join(jieba.cut(x)))

# 对生成的问题进行分词
data["分词问题"] = data["生成的问题"].apply(lambda x: " ".join(jieba.cut(x)))

# 将分词后的问答对转换为适合模型训练的 JSON 格式
train_data = data[["分词问题", "分词答案"]].rename(columns={"分词问题": "question", "分词答案": "answer"})
train_data_list = train_data.to_dict(orient="records")

# 将训练数据保存为 JSON 文件
with open("train_data.json", "w", encoding="utf-8") as f:
    json.dump(train_data_list, f, ensure_ascii=False, indent=4)

print("训练数据已成功保存为 train_data.json 格式（截断处理后）")


训练数据已成功保存为 train_data.json 格式（截断处理后）


#### 1.3正则筛选，去除括号内的数据
效果可行

In [6]:
import pandas as pd
import jieba
import json
import re

# 读取数据
data = pd.read_excel("校园新闻.xlsx", sheet_name="Sheet1")

# 清理空白和重复行
data = data.dropna().drop_duplicates()

# 去除括号及括号内的内容
def remove_parentheses(text):
    # 去除中英文括号内的内容
    return re.sub(r"\(.*?\)|（.*?）", "", text)

# 清理数据中的标题和内容
data["新闻标题"] = data["新闻标题"].apply(remove_parentheses)
data["新闻内容"] = data["新闻内容"].apply(remove_parentheses)

# 基于标题生成简单问题
def generate_question(row):
    title = row["新闻标题"]
    question = f"{title} 是啥情况？"
    return question

# 生成问题列
data["生成的问题"] = data.apply(generate_question, axis=1)

# 截断内容到 "供稿" 之前
def truncate_content(content):
    stop_word = "供稿"
    truncated_content = content.split(stop_word)[0]
    return truncated_content

# 对新闻内容进行截断和分词
data["截断内容"] = data["新闻内容"].apply(truncate_content)
data["分词答案"] = data["截断内容"].apply(lambda x: " ".join(jieba.cut(x)))

# 对生成的问题进行分词
data["分词问题"] = data["生成的问题"].apply(lambda x: " ".join(jieba.cut(x)))

# 将分词后的问答对转换为适合模型训练的 JSON 格式
train_data = data[["分词问题", "分词答案"]].rename(columns={"分词问题": "question", "分词答案": "answer"})
train_data_list = train_data.to_dict(orient="records")

# 将训练数据保存为 JSON 文件
with open("问答对/学校新闻问答对.json", "w", encoding="utf-8") as f:
    json.dump(train_data_list, f, ensure_ascii=False, indent=4)

print("训练数据已成功保存为 学校新闻问答对.json 格式（去除括号内容）")


训练数据已成功保存为 train_data.json 格式（去除括号内容）


#### 1.4 基于时间来分类新闻

In [7]:
import pandas as pd
import jieba
import json
import re
from datetime import datetime

# 读取数据
data = pd.read_excel("校园新闻.xlsx", sheet_name="Sheet1")

# 清理空白和重复行
data = data.dropna().drop_duplicates()

# 去除括号及括号内的内容
def remove_parentheses(text):
    return re.sub(r"\(.*?\)|（.*?）", "", text)

# 清理数据中的标题和内容
data["新闻标题"] = data["新闻标题"].apply(remove_parentheses)

# 提取并格式化发布时间
data["发布日期"] = pd.to_datetime(data["新闻时间"], errors='coerce')  # 将字符串转换为日期格式
data["月份"] = data["发布日期"].dt.strftime("%Y年%m月")  # 提取年月

# 基于发布日期生成问题和答案
def generate_question_answer(row):
    month = row["月份"]
    title = row["新闻标题"]
    question = f"{month} 学校有什么新闻？"
    answer = title  # 回答用标题即可
    return pd.Series([question, answer])

# 生成问答对列
data[["分组问题", "分组答案"]] = data.apply(generate_question_answer, axis=1)

# 对生成的问题进行分词
data["分词问题"] = data["分组问题"].apply(lambda x: " ".join(jieba.cut(x)))
data["分词答案"] = data["分组答案"].apply(lambda x: " ".join(jieba.cut(x)))

# 转换为适合模型训练的 JSON 格式
train_data = data[["分词问题", "分词答案"]].rename(columns={"分词问题": "question", "分词答案": "answer"})
train_data_list = train_data.to_dict(orient="records")

# 保存训练数据为 JSON 文件
with open("问答对/新闻时间分布问答对.json", "w", encoding="utf-8") as f:
    json.dump(train_data_list, f, ensure_ascii=False, indent=4)

print("按月份分类的新闻问答对已成功保存为 新闻时间分布.json ")


按月份分类的新闻问答对已成功保存为 新闻时间分布.json 


#### 1.5 后续的问答对

In [33]:
import pandas as pd
import jieba
import json
import re
from datetime import datetime
excelName = "校园成果和文章汇总.xlsx" #表格名称
sheetName = "校园成果"  # 表格内容
Title = "成果标题"  # 初步问题
Introduce = "具体描述" # 答案
Time = "展示时间" 
# 读取数据
data = pd.read_excel(excelName, sheet_name=sheetName)

# 清理空白和重复行
data = data.dropna().drop_duplicates()
# 清理数据中的标题和内容
data["标题"] = data[Title]
# 通知标题1：为本区域专用的

# 提取并格式化发布时间
data["日期"] = pd.to_datetime(data[Time], errors='coerce')  # 将字符串转换为日期格式
data["月份"] = data["日期"].dt.strftime("%Y年%m月")  # 提取年月

# 基于发布日期生成问题和答案
def generate_question_answer(row):
    month = row["月份"]
    title = row["标题"]
    question = f"{month} 学校有取得什么成果或成绩吗？"
    answer = f"有的，比如：{title}"
    return pd.Series([question, answer])

# 生成问答对列
data[["分组问题", "分组答案"]] = data.apply(generate_question_answer, axis=1)

# 对生成的问题进行分词
data["分词问题"] = data["分组问题"].apply(lambda x: " ".join(jieba.cut(x)))
data["分词答案"] = data["分组答案"].apply(lambda x: " ".join(jieba.cut(x)))

# 转换为适合模型训练的 JSON 格式
train_data = data[["分词问题", "分词答案"]].rename(columns={"分词问题": "question", "分词答案": "answer"})
train_data_list = train_data.to_dict(orient="records")

# 保存训练数据为 JSON 文件
with open("问答对/成果时间分布问答对.json", "w", encoding="utf-8") as f:
    json.dump(train_data_list, f, ensure_ascii=False, indent=4)

print("按月份分类的问答对已成功保存")


按月份分类的问答对已成功保存


#### 1.6 通用问答对儿，自定义问题--答案格式

In [34]:
import pandas as pd
import jieba
import json
import re
excelName = "校园成果和文章汇总.xlsx" #表格名称
sheetName = "校园成果"  # 表格内容
Title = "成果标题"  # 初步问题
Introduce = "具体描述" # 答案

# 读取数据
data = pd.read_excel(excelName, sheet_name=sheetName)

# 清理空白和重复行
data = data.dropna().drop_duplicates()

# 去除括号及括号内的内容
def remove_parentheses(text):
    # 去除中英文括号内的内容
    # return re.sub(r"\(.*?\)|（.*?）", "", text)
    return re.sub(r"\【.*?\】", "", text)

# 清理数据中的标题和内容
# data["标题"] = data[Title].apply(remove_parentheses)
data["内容"] = data[Introduce].apply(remove_parentheses)
# 截断内容到 "" 之前
def truncate_content(content):
    stop_word = "["
    truncated_content = content.split(stop_word)[0]
    return truncated_content

data["标题"] = data[Title]
# data["内容"] = data[Introduce]  
# data["内容"] = data[Introduce].apply(truncate_content)

# 基于发布日期生成问题和答案
def generate_question_answer(row):
    title = row["标题"]
    answer = row["内容"]
    question = f"{title}，展开详细说说？"
    answer = f"以下是获取到的相关信息，{answer}"
    return pd.Series([question, answer])

# 生成问答对列
data[["分组问题", "分组答案"]] = data.apply(generate_question_answer, axis=1)

# 对新闻内容进行截断和分词
# data["截断内容"] = data["分组答案"].apply(truncate_content)
data["截断内容"] = data["分组答案"]
data["分词答案"] = data["截断内容"].apply(lambda x: " ".join(jieba.cut(x)))

# 对生成的问题进行分词
data["分词问题"] = data["分组问题"].apply(lambda x: " ".join(jieba.cut(x)))

# 将分词后的问答对转换为适合模型训练的 JSON 格式
train_data = data[["分词问题", "分词答案"]].rename(columns={"分词问题": "question", "分词答案": "answer"})
train_data_list = train_data.to_dict(orient="records")

# 将训练数据保存为 JSON 文件
with open("问答对/学校成果问答对.json", "w", encoding="utf-8") as f:
    json.dump(train_data_list, f, ensure_ascii=False, indent=4)

print("训练数据已成功保存为 问答对.json 格式")


训练数据已成功保存为 问答对.json 格式


### 2、json转JSONL文件

In [1]:
import json
JSON_Path = "问答对/所有的.json"
# 读取 JSON 文件
with open(JSON_Path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 将数据写入 JSONL 文件
with open("学校数据.jsonl", "w", encoding="utf-8") as f:
    for record in data:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("JSON 文件已成功转换为 JSONL 格式")


JSON 文件已成功转换为 JSONL 格式


#### 2.1内容格式化

In [2]:
import json
JSON_Path = "问答对/所有的.json"
# 读取数据
with open(JSON_Path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 准备存储转换后的数据
formatted_data = []

# 生成所需的结构化数据
for item in data:
    record = {
        "messages": [
            {"role": "system", "content": "河北经贸大学AI助手。"},
            {"role": "user", "content": item["question"]},
            {"role": "assistant", "content": item["answer"]}
        ]
    }
    formatted_data.append(record)

# 保存为 JSONL 文件，每行一条记录
with open("formatted_data.jsonl", "w", encoding="utf-8") as f:
    for record in formatted_data:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("数据已成功转换并保存为 JSONL 格式")


数据已成功转换并保存为 JSONL 格式


## 以下是模型使用

In [12]:
# pip install zhipuai 请先在终端进行安装

from zhipuai import ZhipuAI

client = ZhipuAI(api_key="03a8fac73d39a40ad214d34320e41b1c.KA2l6w0UZwaThLjg")

response = client.chat.completions.create(
    model="glm-4-flash:1851242649:wjs1211:wjyavvqu",
    messages=[
        {
            "role": "system",
            "content": "你是一个乐于解答各种问题的助手，你的任务是为用户提供专业、准确、有见地的建议。" 
        },
        {
            "role": "user",
            "content": "2024 年 10 月   学校 有 什么 通知 ？？"
        }
    ],
    top_p= 0.7,
    temperature= 0.95,
    max_tokens=1024,
    tools = [{"type":"web_search","web_search":{"search_result":True}}],
    stream=False
)

answer_content = response.choices[0].message.content
print("助手回答:", answer_content)
# print(response[0].message.content)
# for trunk in response:
#     print(trunk)

助手回答: 很抱歉，由于我无法访问实时数据或特定学校的通知系统，我无法提供2024年10月某学校具体的通知内容。通常，学校的重要通知会通过官方渠道发布，比如学校网站、校园公告栏、电子邮件或学生管理系统。建议您直接访问您所在学校的官方网站或联系学校相关部门获取最新的通知信息。


In [7]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="03a8fac73d39a40ad214d34320e41b1c.KA2l6w0UZwaThLjg")

# 系统初始化消息
messages = [
    {
        "role": "system",
        "content": "你是一个乐于解答各种问题的助手，你的任务是为用户提供专业、准确、有见地的建议。" 
    }
]

while True:
    # 获取用户输入
    user_input = input("你想问什么？ (输入'退出'结束): ")
    
    if user_input.lower() == '退出':
        print("程序结束。")
        break

    # 将用户输入添加到消息列表
    messages.append({"role": "user", "content": user_input})

    # 调用API获取响应
    response = client.chat.completions.create(
        model="glm-4-flash:1851242649:wjs1211:wjyavvqu",
        messages=messages,
        top_p=0.7,
        temperature=0.95,
        max_tokens=1024,
        tools=[{"type": "web_search", "web_search": {"search_result": True}}],
        # stream=True
        stream=False
    )
    answer = response['choices'][0]['message']['content']
    
    print("助手回答:", answer)

    # 打印每个响应块
    # for trunk in response:
    #     print(trunk)

    # 可以选择将助手的回复添加到消息列表，以便保持上下文
    # messages.append({"role": "assistant", "content": response})


TypeError: 'Completion' object is not subscriptable